This is how we deployed our Keras/TensorFlow into a web app with Docker. Note that `preprocess_one_hotdog.py` is a pre-processing package we wrote to put the test image through InceptionV3 to extract features, before going through our model. Note that this is for binary classification only; we have updated to do multi-class classification, with minor modifications that are not reflected in this markdown.

In [ ]:
import os
import sys

# Flask
from flask import Flask, redirect, url_for, request, render_template, Response, jsonify, redirect
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from keras.models import model_from_json

# Some utilites
import numpy as np
from util import base64_to_pil

# Pre-process image with inceptionnet
import preprocess_one_hotdog

We found that the Js/CSS templates were not linking up upon Docker deployment, so we specifically defined the directories to the js/css templates when delcaring a Flask instance.

In [ ]:
#define js/css templates
TEMPLATE_DIR = os.path.abspath('templates')
STATIC_DIR = os.path.abspath('static')

# app = Flask(__name__) # to make the app run without any
app = Flask(__name__, template_folder=TEMPLATE_DIR, static_folder=STATIC_DIR)

# Declare a flask app
app = Flask(__name__)

Now, we load our best model. We use the `.json` and `.h5` files that we saved previously.

In [ ]:
# Remove src from cwd if necessary
cwd = os.getcwd()
if os.path.basename(cwd) == 'src': cwd = os.path.dirname(cwd)

model_name = 'debugging'

# Create img directory to save images if needed
os.makedirs(os.path.join(cwd, 'img'), exist_ok=True)

# Create model directory to save models if needed
os.makedirs(os.path.join(cwd, 'model'), exist_ok=True)
model_weights_fname = os.path.join(cwd, 'model', model_name + '.h5')
model_json_fname = os.path.join(cwd, 'model', model_name + '.json')

# Load model and its weights
with open(model_json_fname, 'r') as f: model_json = f.read()
model = model_from_json(model_json)
model.load_weights(model_weights_fname)

# Compile model and evaluate its performance on training and test data
model.compile(loss='binary_crossentropy', optimizer='adam',
    metrics=['accuracy'])

#model._make_predict_function()          # Necessary
print('Model loaded. Start serving...')

We write a function to output the correct prediction. We like fUnKy lettering (who doesn't!)

In [ ]:
def model_predict(img, model):
    rounded_predictions = model.predict_classes(img, verbose=1)
    if rounded_predictions[0] == 0:
        prediction = 'HOtDoG'
    else:
        prediction = 'nOThOtDoG'
    return prediction

The following code discusses app GET/POST methods, to extract information from the drag/drop box. This code is mainly untouched except for the image preprocessing parts.

In [ ]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

In [ ]:
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        # Get the image from post request
        img = base64_to_pil(request.json)

        # Save the image to ./uploads
        img.save(os.path.join(cwd, "demo/test/1.png"))
        print(os.path.join(cwd, "demo/test/1.png"))
        
        #preprocess image with inceptionNetV3
        #use script preprocess_one_hotdog, which should be in same directory
        img = preprocess_one_hotdog.get_single_image_data()
        
        # Make prediction
        preds = model_predict(img, model)

        return jsonify(
            result=preds,
        )

    return None

In [ ]:
if __name__ == '__main__':
    # app.run(port=5002, threaded=False)

    # Serve the app with gevent
    http_server = WSGIServer(('0.0.0.0', 5000), app)
    http_server.serve_forever()

**Authors**: Victoria Liu and Gloria Liu

**Last modified**: November 2020

Description: A script to deploy Keras into a web app with Flask

**Credits**: The base was taken from [open source](https://github.com/mtobeiyf/keras-flask-deploy-webapp) and then heavily modified to deploy our particular model. The requirements.txt files were also changed to match our dependencies. Changes are discussed in the Markdown. The HTML/CSS/JS files (not shown in this notebook) were modified for decoration purposes.